# QUICK DATA CHECK

* Make sure that you have access to data files (have a look at some examples in `/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-II/PDS_Commissioning/waffles/0_TUTORIAL`)

* If you want to analyze new data, you need to pre-process the decoder first to extract PDS information from the raw `hdf5` file provided by the data acquisition system. Have a look at the `00_HDF5toROOT` scripts for this.



In [ ]:
# IMPORT ALL THE LIBRARIES USED IN THE NOTEBOOK
import plotly.graph_objects as pgo
import plotly.subplots as psu
import numpy as np
from APAMaps import apas_map

# IMPORT ALL THE CLASSES/METHODS DEFINED IN WAFFLES
import waffles

In [ ]:
filepath = '/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-II/PDS_Commissioning/waffles/0_TUTORIAL/run26687.root'

wfset = waffles.WaveformSet.from_ROOT_file( filepath,                               # path to the root file 
                                            bulk_data_tree_name = 'raw_waveforms',  # 'raw_waveforms' branch from the root file
                                            meta_data_tree_name = 'metadata',       # 'metadata' branch from the root file
                                            set_offset_wrt_daq_window = False,      #
                                            read_full_streaming_data = False,       # self-triggered (False) data
                                            truncate_wfs_to_minimum = False,        # truncate the waveforms to the minimum size
                                            start_fraction = 0.0,                   # starting fraction for reading
                                            stop_fraction = 1.0,                    # stoping fraction for reading
                                            library = 'pyroot',                     # library to read (if ROOT, use 'pyroot', if not `uproot`)
                                            subsample = 2)                          # subsample the data reading (read each 2 entries)
              

# filtered_wfset = WaveformSet.from_filtered_WaveformSet( mywvfset,
#                                                         WaveformSet.match_endpoint_and_channel,
#                                                         111,
#                                                         46)
# filtered_wfset.Waveforms[i].Adcs

In [ ]:
# You can have a look on the attributes of the object mywvfset (or any other object) by typing:

help(wfset) # Help on the object and displays the documentation

In [ ]:
# A naive example of how to access the wvfset attributes:

print(f"First wvf ADCs: {wfset.Waveforms[0].Adcs}")
print(f"\nNumber of points per wvf: {wfset.PointsPerWf}")
print(f"\nRuns: {wfset.Runs}")
print(f"\nRecords numbers per run(~events numbers): {wfset.RecordNumbers}")
print(f"\nChannels with available data per endpoint: {wfset.AvailableChannels}")

In [ ]:
# You can check which APA data is available in the waveformset by checking the keys of the dictionary:

set_of_endpoints = wfset.get_set_of_endpoints()
apas = []

if 104 in set_of_endpoints or 105 in set_of_endpoints or 107 in set_of_endpoints:
    print("There is APA 1 info. in this waveformset.")
    apas.append('apa_1')
if 109 in set_of_endpoints:
    print("There is APA 2 info. in this waveformset.")
    apas.append('apa_2')
if 111 in set_of_endpoints:
    print("There is APA 3 info. in this waveformset.")
    apas.append('apa_3')
if 112 in set_of_endpoints or 113 in set_of_endpoints:
    print("There is APA 4 info. in this waveformset.")
    apas.append('apa_4')

In [ ]:
grid_apa_3 = waffles.ChannelWSGrid( apas_map['apa_3'],
                                    wfset,
                                    compute_calib_histo = False,
                                    bins_number = 115,
                                    domain = np.array((-10000., 50000.)),
                                    variable = 'integral',
                                    analysis_label = '')

grid_apa_4 = waffles.ChannelWSGrid( apas_map['apa_4'],
                                    wfset,
                                    compute_calib_histo = False,
                                    bins_number = 115,
                                    domain = np.array((-10000., 50000.)),
                                    variable = 'integral',
                                    analysis_label = '')

In [ ]:
title = 'APA3'

figure = psu.make_subplots( rows = 10,
                            cols = 4)

grid_apa_3.plot(figure = figure,
                share_x_scale = False,
                share_y_scale = False,
                mode = 'overlay',
                wfs_per_axes = 2)

figure.update_layout(   title = {
                                    'text': title,
                                    'font': {
                                                'size': 24  # Specify the font size for the title
                                            }
                                },
                        width = 1100,
                        height = 1200,
                        showlegend = True)

figure.show()

# figure.write_image(f"{saving_filepath}/apa_3_NAME.png")

In [ ]:
title = 'APA4'

figure = psu.make_subplots( rows = 10,
                            cols = 4)

grid_apa_4.plot(figure = figure,
                share_x_scale = False,
                share_y_scale = False,
                mode = 'overlay',
                wfs_per_axes = 2)

figure.update_layout(   title = {
                                    'text': title,
                                    'font': {
                                                'size': 24  # Specify the font size for the title
                                            }
                                },
                        width = 1100,
                        height = 1200,
                        showlegend = True)

figure.show()

# figure.write_image(f"{saving_filepath}/apa_4_NAME.png")